In [1]:
import pandas as pd
import string
from sklearn.pipeline import Pipeline
import numpy as np


In [2]:
df_review = pd.read_excel("/content/zenius_data.xlsx")
df_review.head(5)

,userName,ranting,tanggal,ulasan
0,Dewi Adikara,5,2022-06-27 13:20:04,"Dewi udah cobain Zenius, menyenangkan banget c..."
1,KEYSHA DWI SETIYANI,5,2022-06-27 12:57:24,pembelajaran seru dan mudahh dipahami
2,Sri Wahyuni,5,2022-06-27 05:58:22,Aplikasi Zenius sangat membantu adikku untuk b...
3,D.A.P Official,5,2022-06-27 05:41:55,Ada kurikulum baru kurikulum 2022 Implementasi...
4,Nabila Albatul,1,2022-06-27 04:59:09,Hai Zenius aku akan memberi 5 bintang nanti ka...


#jumlah rating

In [3]:
df_review["ranting"].value_counts()

5    887
4    242
3    159
1    130
2     82
Name: ranting, dtype: int64

2.plabelan

In [4]:
sentimen = []
for index, row in df_review.iterrows():
     if row['ranting']>3:
        sentimen.append('positif')
     else:
       sentimen.append('negatif')
df_review['sentimen'] = sentimen
df_review = df_review.drop(columns=['ranting'])
df_review.head()

,userName,tanggal,ulasan,sentimen
0,Dewi Adikara,2022-06-27 13:20:04,"Dewi udah cobain Zenius, menyenangkan banget c...",positif
1,KEYSHA DWI SETIYANI,2022-06-27 12:57:24,pembelajaran seru dan mudahh dipahami,positif
2,Sri Wahyuni,2022-06-27 05:58:22,Aplikasi Zenius sangat membantu adikku untuk b...,positif
3,D.A.P Official,2022-06-27 05:41:55,Ada kurikulum baru kurikulum 2022 Implementasi...,positif
4,Nabila Albatul,2022-06-27 04:59:09,Hai Zenius aku akan memberi 5 bintang nanti ka...,negatif


In [5]:
df_review['sentimen'].value_counts()

positif    1129
negatif     371
Name: sentimen, dtype: int64

3.menghilangkan tabel yang tidak digunakan

In [6]:
df_preprocessed = df_review.copy()
df_preprocessed = df_preprocessed.drop (columns=['userName', 'tanggal'])
df_preprocessed.head()

,ulasan,sentimen
0,"Dewi udah cobain Zenius, menyenangkan banget c...",positif
1,pembelajaran seru dan mudahh dipahami,positif
2,Aplikasi Zenius sangat membantu adikku untuk b...,positif
3,Ada kurikulum baru kurikulum 2022 Implementasi...,positif
4,Hai Zenius aku akan memberi 5 bintang nanti ka...,negatif


3.data disama ratakan(resempling)

In [7]:
from dataclasses import replace
positif = df_preprocessed[df_preprocessed['sentimen']=='positif'].sample(   371 , replace=True)
negatif = df_preprocessed[df_preprocessed['sentimen']=='negatif'].sample(     371 , replace=True)
df_preprocessed = pd.concat([positif, negatif])

print(df_preprocessed.shape)
print(df_preprocessed['sentimen'].value_counts(normalize = True))

(742, 2)
positif    0.5
negatif    0.5
Name: sentimen, dtype: float64


#cleansing

In [8]:
import string, re

def cleansing(data):
  #lower text
  data = data.lower()

  #hapus punctuation
  remove =string.punctuation
  translator = str.maketrans(remove,' '*len(remove))
  data = data.translate(translator)

  #remove ASCII dan unicode
  data = data.encode('ascii','ignore').decode('utf-8')
  data = re.sub(r'[^\x00-\x7f]',r'', data)

  #remove newline
  data = data.replace('\n','')

  return data

  

In [9]:
#jalankan cleansing data
review =[]
for index, row in df_preprocessed.iterrows():
    review.append(cleansing(row["ulasan"]))

df_preprocessed["ulasan"] = review
df_preprocessed.head()


,ulasan,sentimen
1022,apknya diupdate dong ditambahin fitur yg bisa ...,positif
571,halo saya pengguna zenius saya beri rating 5 ...,positif
789,terimakasih sangat membantu untuk generasi in...,positif
632,aplikasi edukasi paling the best konsepnya mu...,positif
1039,terimakasih sebesar besarnya untuk yang mencip...,positif


#remove stopword

In [10]:
!pip install Sastrawi


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 5.2 MB/s 


In [11]:
# import libary
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

#contoh
kalimat ='Aplikasi yg keren nih, khususnya untuk fitur terbaru ZenCore.. Kita bisa melatih kemampuan belajar di CorePractice berkali-kali.. Belajar jadi lebih asik dan nyaman.. Semoga kedepannya muncul fitur" keren lagi ya'
stop = stopword.remove(kalimat)
print(stop)


Aplikasi yg keren nih, khususnya fitur terbaru ZenCore.. Kita melatih kemampuan belajar CorePractice berkali-kali.. Belajar jadi lebih asik nyaman.. Semoga kedepannya muncul fitur" keren ya


In [12]:
review = []
for index, row in df_preprocessed.iterrows():
     review.append(stopword.remove(row["ulasan"]))

df_preprocessed["ulasan"] = review
df_preprocessed.head()

,ulasan,sentimen
1022,apknya diupdate dong ditambahin fitur yg pilih...,positif
571,halo pengguna zenius beri rating 5 dengan zen...,positif
789,terimakasih sangat membantu generasi indonesi...,positif
632,aplikasi edukasi paling the best konsepnya mu...,positif
1039,terimakasih sebesar besarnya yang menciptakan ...,positif


#steaming

In [13]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
Stemmer = factory.create_stemmer()
#contoh
kalimat = 'Aplikasi yg keren nih, khususnya untuk fitur terbaru ZenCore.. Kita bisa melatih kemampuan belajar di CorePractice berkali-kali.. Belajar jadi lebih asik dan nyaman.. Semoga kedepannya muncul fitur" keren lagi ya'
katadasar = Stemmer.stem(kalimat)
print(katadasar)

aplikasi yg keren nih khusus untuk fitur baru zencore kita bisa latih mampu ajar di corepractice kali ajar jadi lebih asik dan nyaman moga depan muncul fitur keren lagi ya


In [15]:
#implementasi pada data kita
review = []
for index, row in df_preprocessed.iterrows():
     review.append(Stemmer.stem(row["ulasan"]))

df_preprocessed["ulasan"] = review
df_preprocessed.head()

,ulasan,sentimen
1022,apknya diupdate dong ditambahin fitur yg pilih...,positif
571,halo guna zenius beri rating 5 dengan zenius p...,positif
789,terimakasih sangat bantu generasi indonesia yg...,positif
632,aplikasi edukasi paling the best konsep mudah ...,positif
1039,terimakasih besar besar yang cipta aplikasi ze...,positif


In [16]:
df_preprocessed.to_excel("zen.xlsx", index = False)  #Save the file as CSV , to download: click the folder icon on the left. the csv file should be there.

menbagi data train dan data test

In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test = train_test_split(df_preprocessed['ulasan'],df_preprocessed['sentimen'],
                                                   test_size=0.2, stratify=df_preprocessed['sentimen'], random_state=0)

In [18]:
x_train


391     video materi nya baguss bgt kenapa apk nya lel...
908     aplikasi ajar paling the best lah ndapat kuota...
12      axis harap jangan download gak di voucher nyes...
965     bagus sih cuma jelas tuh tele tele gak to the ...
1358    aplikasi sangat bagus cocok siswa i yg ikut bi...
                              ...                        
174     ku sudah bagus kenapa kalau latih soal zencore...
371                      aplikasi nya terkadang bisa buka
249     aplikasi nya padahal uda bagus minus video aja...
1499    ui ux nya minimal simple bagus jadi lebih enak...
1368    jujur aplikasi zenius nih bagus banget lho saa...
Name: ulasan, Length: 593, dtype: object

In [19]:
x_test

790     udah bagus sih cuman kadang video yg dikit ban...
495     akhir akhir agak kecewa sebenernya udah 1 bula...
741     makin lama versi web makin kacau banyak bug la...
539     zenius bagus materi tapiii ya sering banget ka...
1322    aplikasi bagus jelas video jelas tetapi bebera...
                              ...                        
63      materi mudah erti guru asik jadi ga bosen buat...
419     fitur ganti tingkat kelas mana sih bagus tampi...
1057    maaf belum ka ko filter uduh video yak adain a...
1011    hatur nuhun app ajar sangat sangat sekali bant...
443     aplikasi nya sering ngebug di buat download vidoe
Name: ulasan, Length: 149, dtype: object

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(x_train)

TfidfVectorizer()

In [21]:
vectorizer.vocabulary_

{'video': 1993,
 'materi': 1098,
 'nya': 1309,
 'baguss': 165,
 'bgt': 247,
 'kenapa': 853,
 'apk': 113,
 'lelet': 986,
 'sih': 1691,
 'kesel': 869,
 'mau': 1103,
 'nonton': 1296,
 'susah': 1776,
 'loading': 1011,
 'lama': 956,
 'padahal': 1365,
 'sinyal': 1704,
 'internet': 718,
 'aman': 83,
 'jaya': 753,
 'lancar': 962,
 'ajar': 56,
 'jadi': 733,
 'gk': 607,
 'mood': 1174,
 'nunggu': 1306,
 'tahun': 1784,
 'mohon': 1172,
 'diperbaikin': 453,
 'sudah': 1763,
 'uldate': 1943,
 'kali': 804,
 'versi': 1987,
 'yg': 2052,
 'baru': 194,
 'aplikasi': 116,
 'paling': 1377,
 'the': 1861,
 'best': 242,
 'lah': 950,
 'ndapat': 1215,
 'kuota': 940,
 'gratis': 616,
 'edukasi': 493,
 'gue': 624,
 'sering': 1673,
 'buka': 312,
 'buat': 303,
 'siap': 1687,
 'uji': 1938,
 'jamin': 740,
 'langsung': 966,
 'paham': 1369,
 'deh': 402,
 'terimakasih': 1842,
 'developer': 419,
 'axis': 153,
 'harap': 650,
 'jangan': 741,
 'download': 477,
 'gak': 571,
 'di': 426,
 'voucher': 2005,
 'nyesekkk': 1322,
 'bagu

In [22]:
#pada data kita
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

print(x_test.shape)
print(x_test.shape)

(149, 2080)
(149, 2080)


In [23]:
x_train.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

#preproses data

In [24]:
def preprocess_data(data):
  # cleansing data
  data = cleansing(data)

  #hapus stopwords
  factory = StopWordRemoverFactory()
  stopword = factory.create_stop_word_remover()
  data = stopword.remove(data)

  #steming
  factory = StemmerFactory()
  Stemmer = factory.create_stemmer()
  data = Stemmer.stem(data)

  # count vectorizer
  data = vectorizer.transform([data])
  return data

proses svm

In [25]:
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
for c in [0.01, 0.05, 0.25, 0.5, 0.75, 1]:
    svm = LinearSVC(C=c)
    svm.fit(x_train,y_train)
    print('Akurasi untuk c = %s: %s' %(c, accuracy_score(y_test, svm.predict(x_test))))

Akurasi untuk c = 0.01: 0.8120805369127517
Akurasi untuk c = 0.05: 0.8389261744966443
Akurasi untuk c = 0.25: 0.8456375838926175
Akurasi untuk c = 0.5: 0.825503355704698
Akurasi untuk c = 0.75: 0.825503355704698
Akurasi untuk c = 1: 0.8120805369127517


In [26]:
#final model
svm = LinearSVC(C=0.75)
svm.fit(x_train, y_train)

LinearSVC(C=0.75)

In [27]:
print('Accuracy score model final: %s ' %accuracy_score(y_test, svm.predict(x_test)))

Accuracy score model final: 0.825503355704698 
